In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import os
import cv2
import numpy as np
import os, re, glob
import cv2
from tqdm import tqdm
import numpy as np

In [ ]:
PATH_tablet = '../gdrive/MyDrive/공모전용/HW-OCR/4.Validaiton/tablet/R.Free/전처리/'

In [ ]:
categories = os.listdir(PATH_tablet)

In [ ]:
num_classes = len(categories)
num_classes

968

In [ ]:
X = []
Y = []
count = 0

In [ ]:
image_w = 48
image_h = 48

for idex, categorie in tqdm(enumerate(categories)):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = PATH_tablet + categorie + '/'
  
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            count+=1
            img = cv2.imread(image_dir+filename)
            try:
              img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            except:
              print('에러명 : ',image_dir+filename)
              continue
            img = cv2.resize(img, dsize=(48, 48))
            X.append(img/256)
            Y.append(label)
            
 
X = np.array(X)
Y = np.array(Y)


40it [12:48,  7.94s/it]

에러명 :  ../gdrive/MyDrive/공모전용/HW-OCR/4.Validaiton/tablet/R.Free/전처리/상/265431_상.png


256it [44:50, 10.19s/it]

에러명 :  ../gdrive/MyDrive/공모전용/HW-OCR/4.Validaiton/tablet/R.Free/전처리/시/264433_시.png


968it [1:40:04,  6.20s/it]


## 모델링 & Tensorboard

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
xy = (X_train, X_test, Y_train, Y_test)

In [ ]:
import numpy as np

In [ ]:
## 모델 저장. data_1은 tablet - R 데이터 뿐
np.save('../gdrive/MyDrive/공모전용/HW-OCR/data_1.npy',xy)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense
from keras.layers import Flatten, Convolution2D, MaxPooling2D
from keras.models import load_model
import cv2
import numpy as np

In [ ]:
#X_train, X_test, Y_train, Y_test = np.load('data_1.npy')

FileNotFoundError: ignored

In [ ]:
X_train = X_train.reshape((142719, 48, 48, 1))
X_test = X_test.reshape((47574, 48, 48, 1))

In [ ]:
from keras import models, layers

In [ ]:

CNN = models.Sequential()
CNN.add(layers.Conv2D(128, (3,3), activation='relu',input_shape=(48,48,1)))
CNN.add(layers.MaxPooling2D((2,2)))
CNN.add(layers.Dropout(0.25))

CNN.add(layers.Conv2D(256, (3,3), activation='relu'))
CNN.add(layers.MaxPooling2D((2,2)))
CNN.add(layers.Dropout(0.25))

CNN.add(layers.Conv2D(512, (3,3), activation='relu'))
CNN.add(layers.Flatten())

CNN.add(layers.Dense(512, activation='relu'))

CNN.add(layers.Dense(num_classes, activation='softmax'))

#CNN 함수 설정 후 Training
from keras import optimizers
CNN.compile(optimizer = 'adam',
loss = 'categorical_crossentropy',
metrics = ['accuracy'])

CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 128)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 256)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 256)       0

In [ ]:
hist = CNN.fit(X_train, Y_train,
epochs = 10, batch_size = 128,
validation_split=0.2)
# model.save('Gersang.h5')

Epoch 1/10
892/892 [==============================] - 35s 30ms/step - loss: 3.5425 - accuracy: 0.3324 - val_loss: 1.4871 - val_accuracy: 0.6651
Epoch 2/10
892/892 [==============================] - 25s 28ms/step - loss: 1.0207 - accuracy: 0.7498 - val_loss: 0.7792 - val_accuracy: 0.8105
Epoch 3/10
892/892 [==============================] - 25s 28ms/step - loss: 0.5725 - accuracy: 0.8436 - val_loss: 0.6349 - val_accuracy: 0.8393
Epoch 4/10
892/892 [==============================] - 25s 28ms/step - loss: 0.3893 - accuracy: 0.8858 - val_loss: 0.6269 - val_accuracy: 0.8442
Epoch 5/10
892/892 [==============================] - 25s 28ms/step - loss: 0.2825 - accuracy: 0.9138 - val_loss: 0.5844 - val_accuracy: 0.8637
Epoch 6/10
892/892 [==============================] - 25s 28ms/step - loss: 0.2216 - accuracy: 0.9302 - val_loss: 0.6121 - val_accuracy: 0.8651
Epoch 7/10
892/892 [==============================] - 25s 28ms/step - loss: 0.1781 - accuracy: 0.9423 - val_loss: 0.6573 - val_accuracy: